In [2]:
import  pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import  plotly.express as px


In [6]:
from core import startproject as start

In [7]:
dataset_dir = start.init()

<br>**PROJECT 2022 - EMPLOYEE TURNOVER**

Version              : 1.0.0
Notebook id          : EMPLOYEETURNOVER
Run time             : Saturday 05 November 2022, 17:08:35
Hostname             : kaliAlphadeep (Linux)
Tensorflow log level : Info + Warning + Error  (=0)
Datasets dir         : /data/AllProjectIA/EmployeesTurnover/dataset
Run dir              : ./run
tensorflow           : 2.10.0
numpy                : 1.23.4
sklearn              : 1.1.3
matplotlib           : 3.6.2
plotly               : 5.11.0
pandas               : 1.5.1


# get Data

## get fake data of kaggle

In [9]:
data1 = pd.read_csv("dataset/data1/WA_Fn-UseC_-HR-Employee-Attrition.csv")

In [10]:
data2 = pd.read_csv("dataset/data2/MFG10YearTerminationData.csv")

In [12]:
data3 = pd.read_csv("dataset/data3/turnover.csv",encoding='latin1')

## get data from formular based on system of number of like and dislike

In [18]:
churn = pd.read_csv("dataset/data4/churn.csv")
comment_interaction = pd.read_csv("dataset/data4/commentInteractions.csv")
vote = pd.read_csv("dataset/data4/votes.csv")
comment_anonyme = pd.read_csv("dataset/data4/comments_clean_anonimized.csv")

/tmp/ipykernel_117923/2299788218.py:2: DtypeWarning: Columns (2,3) have mixed types. Specify dtype option on import or set low_memory=False.
  comment_interaction = pd.read_csv("dataset/data4/commentInteractions.csv")


In [19]:
#remove duplicates row
churn = churn.drop_duplicates()
comment_anonyme = comment_anonyme.drop_duplicates()
comment_interaction = comment_interaction.drop_duplicates()
vote = vote.drop_duplicates()

In [21]:
# merge data
merged = churn.merge(vote, left_on=['employee', 'companyAlias'], right_on=['employee','companyAlias'])
merged = merged.drop(['lastParticipationDate', 'voteDate'], 1)
merged

/tmp/ipykernel_117923/2264114485.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  merged = merged.drop(['lastParticipationDate', 'voteDate'], 1)


,employee,companyAlias,numVotes,stillExists,vote
0,512,56aec740f1ef260003e307d6,4,True,3
1,512,56aec740f1ef260003e307d6,4,True,2
2,512,56aec740f1ef260003e307d6,4,True,1
3,512,56aec740f1ef260003e307d6,4,True,4
4,2,56aec740f1ef260003e307d6,72,True,2
...,...,...,...,...,...
222878,17,573a0671b5ec330003add34a,7,True,3
222879,17,573a0671b5ec330003add34a,7,True,4
222880,17,573a0671b5ec330003add34a,7,True,4
222881,17,573a0671b5ec330003add34a,7,True,4


In [22]:
# group merge by employee and company
m = merged.groupby(['employee', 'companyAlias']).agg(numVotes=('numVotes', 'mean'), vote=('vote', 'mean'), stillExists=('stillExists', np.all))
m

numVotes      vote  stillExists
employee companyAlias                                             
1        5474b9cde4b0bf7614b2c66f       2.0  4.000000         True
         54d43612e4b0f6a40755d93e      16.0  3.125000        False
         54e52607e4b01191dc064966     135.0  2.985075        False
         5641f96713664c000332c8cd     257.0  3.171206         True
         56558cfd07a5de00030908fb       2.0  3.500000         True
...                                     ...       ...          ...
991      54e52607e4b01191dc064966       1.0  3.000000         True
995      54e52607e4b01191dc064966       6.0  2.666667         True
996      54e52607e4b01191dc064966      20.0  2.450000         True
997      54e52607e4b01191dc064966       1.0  3.000000        False
999      54e52607e4b01191dc064966      20.0  2.850000         True

[4377 rows x 3 columns]

In [23]:
# generate unique id
merged['employee'] = merged['employee'].apply(str)
merged['combined_id'] = pd.factorize(merged.employee+merged.companyAlias)[0]
merged = merged.drop(['employee', 'companyAlias'], 1)
merged

/tmp/ipykernel_117923/2617863580.py:4: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  merged = merged.drop(['employee', 'companyAlias'], 1)


,numVotes,stillExists,vote,combined_id
0,4,True,3,0
1,4,True,2,0
2,4,True,1,0
3,4,True,4,0
4,72,True,2,1
...,...,...,...,...
222878,7,True,3,4375
222879,7,True,4,4375
222880,7,True,4,4375
222881,7,True,4,4375


In [25]:
#group now by unique id
m = merged.groupby('combined_id').agg(numVotes=('numVotes', 'mean'), vote=('vote', 'mean'), stillExists=('stillExists', np.all))

In [26]:
m.reset_index()
m.head()

,numVotes,vote,stillExists
combined_id,,,
0,4.0,2.500000,True
1,72.0,2.250000,True
2,14.0,3.142857,False
3,22.0,3.454545,True
4,195.0,3.856410,True


In [29]:
m.to_csv('dataset/data4/data4.csv', index=False)